<a href="https://colab.research.google.com/github/eldercamposds/AnaliseDeSentimento/blob/SupervisionadoVSRegras/SupervisionadoVsRegras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score


from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
tweets = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP/7.Analise de sentimentos/Tweets2.csv")

In [ ]:
tweets.shape

(73996, 4)

In [ ]:
tweets.head()

,id,local,sentiment,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
tweets.groupby(["sentiment"]).size()

,0
sentiment,
Irrelevant,12990
Negative,22542
Neutral,18318
Positive,20832


In [ ]:
tweets.loc[tweets.sentiment == 'Irrelevant', "sentiment"] = "Neutral"

In [ ]:
tweets.groupby(["sentiment"]).size()

,0
sentiment,
Negative,22542
Neutral,31308
Positive,20832


In [ ]:
tweets = tweets.dropna(subset=["text"])
tweets.reset_index(drop=True, inplace=True)

In [ ]:
tweets.groupby(["sentiment"]).size()

,0
sentiment,
Negative,22358
Neutral,30983
Positive,20655


#Supervisionado